# Importando dependências

In [2]:
import findspark

In [3]:
# Iniciando conexão com spark (o parâmetro de localização do spark pode ser omitido)
findspark.init('/usr/local/spark')

# Criando uma sessão do Spark

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .master('local') \
        .appName('learning-spark') \
        .config('spark.executor.memory', '1gb') \
        .getOrCreate()

sc = spark.sparkContext

# Lendo arquivos

In [5]:
# Definindo os dados crús
rdd = sc.textFile('CaliforniaHousing/cal_housing.data')

# Definindo o cabeçalho
header = sc.textFile('CaliforniaHousing/cal_housing.domain')

In [6]:
# Somente com o método .collect() o arquivo é lido
header.collect()

['longitude: continuous.',
 'latitude: continuous.',
 'housingMedianAge: continuous. ',
 'totalRooms: continuous. ',
 'totalBedrooms: continuous. ',
 'population: continuous. ',
 'households: continuous. ',
 'medianIncome: continuous. ',
 'medianHouseValue: continuous. ']

In [7]:
# Também podemos utilizao o .take(n), onde n = número de registros à serem lidos
rdd.take(5)

['-122.230000,37.880000,41.000000,880.000000,129.000000,322.000000,126.000000,8.325200,452600.000000',
 '-122.220000,37.860000,21.000000,7099.000000,1106.000000,2401.000000,1138.000000,8.301400,358500.000000',
 '-122.240000,37.850000,52.000000,1467.000000,190.000000,496.000000,177.000000,7.257400,352100.000000',
 '-122.250000,37.850000,52.000000,1274.000000,235.000000,558.000000,219.000000,5.643100,341300.000000',
 '-122.250000,37.850000,52.000000,1627.000000,280.000000,565.000000,259.000000,3.846200,342200.000000']

In [8]:
# Dividindo os campos de cada linha
rdd = rdd.map(lambda line: line.split(','))

rdd.take(2)

[['-122.230000',
  '37.880000',
  '41.000000',
  '880.000000',
  '129.000000',
  '322.000000',
  '126.000000',
  '8.325200',
  '452600.000000'],
 ['-122.220000',
  '37.860000',
  '21.000000',
  '7099.000000',
  '1106.000000',
  '2401.000000',
  '1138.000000',
  '8.301400',
  '358500.000000']]

In [9]:
# Recuperando a primeira linha
rdd.first()

['-122.230000',
 '37.880000',
 '41.000000',
 '880.000000',
 '129.000000',
 '322.000000',
 '126.000000',
 '8.325200',
 '452600.000000']

In [10]:
# Mesmo que .take(n)
rdd.top(2)

[['-124.350000',
  '40.540000',
  '52.000000',
  '1820.000000',
  '300.000000',
  '806.000000',
  '270.000000',
  '3.014700',
  '94600.000000'],
 ['-124.300000',
  '41.840000',
  '17.000000',
  '2677.000000',
  '531.000000',
  '1244.000000',
  '456.000000',
  '3.031300',
  '103600.000000']]

# Convertendo RDD em DataFrame

In [11]:
# Importando dependencia 
from pyspark.sql import Row

# Mapeando o RDD para DataFrame
df = rdd.map(lambda line: Row(longitude=line[0],
                                 latitude=line[1],
                                 housingMedianAge=line[2],
                                 totalRooms=line[3],
                                 totalBedrooms=line[4],
                                 population=line[5],
                                 households=line[6],
                                 medianIncome=line[7],
                                 medianHouseValue=line[8])).toDF()

In [12]:
df.head(5)

[Row(longitude='-122.230000', latitude='37.880000', housingMedianAge='41.000000', totalRooms='880.000000', totalBedrooms='129.000000', population='322.000000', households='126.000000', medianIncome='8.325200', medianHouseValue='452600.000000'),
 Row(longitude='-122.220000', latitude='37.860000', housingMedianAge='21.000000', totalRooms='7099.000000', totalBedrooms='1106.000000', population='2401.000000', households='1138.000000', medianIncome='8.301400', medianHouseValue='358500.000000'),
 Row(longitude='-122.240000', latitude='37.850000', housingMedianAge='52.000000', totalRooms='1467.000000', totalBedrooms='190.000000', population='496.000000', households='177.000000', medianIncome='7.257400', medianHouseValue='352100.000000'),
 Row(longitude='-122.250000', latitude='37.850000', housingMedianAge='52.000000', totalRooms='1274.000000', totalBedrooms='235.000000', population='558.000000', households='219.000000', medianIncome='5.643100', medianHouseValue='341300.000000'),
 Row(longitude

In [13]:
df.show(3)

+-----------+---------+----------------+-----------+-------------+-----------+-----------+------------+----------------+
|  longitude| latitude|housingMedianAge| totalRooms|totalBedrooms| population| households|medianIncome|medianHouseValue|
+-----------+---------+----------------+-----------+-------------+-----------+-----------+------------+----------------+
|-122.230000|37.880000|       41.000000| 880.000000|   129.000000| 322.000000| 126.000000|    8.325200|   452600.000000|
|-122.220000|37.860000|       21.000000|7099.000000|  1106.000000|2401.000000|1138.000000|    8.301400|   358500.000000|
|-122.240000|37.850000|       52.000000|1467.000000|   190.000000| 496.000000| 177.000000|    7.257400|   352100.000000|
+-----------+---------+----------------+-----------+-------------+-----------+-----------+------------+----------------+
only showing top 3 rows



In [14]:
df.first()

Row(longitude='-122.230000', latitude='37.880000', housingMedianAge='41.000000', totalRooms='880.000000', totalBedrooms='129.000000', population='322.000000', households='126.000000', medianIncome='8.325200', medianHouseValue='452600.000000')

In [15]:
# Colunas
df.columns

['longitude',
 'latitude',
 'housingMedianAge',
 'totalRooms',
 'totalBedrooms',
 'population',
 'households',
 'medianIncome',
 'medianHouseValue']

In [16]:
# Schema
df.printSchema()

root
 |-- longitude: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- housingMedianAge: string (nullable = true)
 |-- totalRooms: string (nullable = true)
 |-- totalBedrooms: string (nullable = true)
 |-- population: string (nullable = true)
 |-- households: string (nullable = true)
 |-- medianIncome: string (nullable = true)
 |-- medianHouseValue: string (nullable = true)



In [17]:
# Convertendo dados em float para melhorara o desempenho do DataFrame
from pyspark.sql.types import *

df = df.withColumn('longitude', df['longitude'].cast(FloatType())) \
        .withColumn('latitude', df['latitude'].cast(FloatType())) \
        .withColumn('housingMedianAge', df['housingMedianAge'].cast(FloatType())) \
        .withColumn('totalRooms', df['totalRooms'].cast(FloatType())) \
        .withColumn('totalBedrooms', df['totalBedrooms'].cast(FloatType())) \
        .withColumn('population', df['population'].cast(FloatType())) \
        .withColumn('households', df['households'].cast(FloatType())) \
        .withColumn('medianIncome', df['medianIncome'].cast(FloatType())) \
        .withColumn('medianHouseValue', df['medianHouseValue'].cast(FloatType()))

In [18]:
# Convertendo dados em float com uma função
def convertColumnsToFloat(df, columns):
    for col in columns:
        df = df.withColumn(col, df[col].cast(FloatType()))
    return df

# Converter todas as colunas
columns_to_convert = df.columns

# Aplicando a função
df = convertColumnsToFloat(df, columns_to_convert)

In [19]:
df.printSchema()

root
 |-- longitude: float (nullable = true)
 |-- latitude: float (nullable = true)
 |-- housingMedianAge: float (nullable = true)
 |-- totalRooms: float (nullable = true)
 |-- totalBedrooms: float (nullable = true)
 |-- population: float (nullable = true)
 |-- households: float (nullable = true)
 |-- medianIncome: float (nullable = true)
 |-- medianHouseValue: float (nullable = true)



In [20]:
# Selecionando collunas
df.select('population', 'totalBedrooms').show(10)

+----------+-------------+
|population|totalBedrooms|
+----------+-------------+
|     322.0|        129.0|
|    2401.0|       1106.0|
|     496.0|        190.0|
|     558.0|        235.0|
|     565.0|        280.0|
|     413.0|        213.0|
|    1094.0|        489.0|
|    1157.0|        687.0|
|    1206.0|        665.0|
|    1551.0|        707.0|
+----------+-------------+
only showing top 10 rows



In [21]:
df.groupBy('housingMedianAge').count().sort('housingMedianAge', ascending=False).show()

+----------------+-----+
|housingMedianAge|count|
+----------------+-----+
|            52.0| 1273|
|            51.0|   48|
|            50.0|  136|
|            49.0|  134|
|            48.0|  177|
|            47.0|  198|
|            46.0|  245|
|            45.0|  294|
|            44.0|  356|
|            43.0|  353|
|            42.0|  368|
|            41.0|  296|
|            40.0|  304|
|            39.0|  369|
|            38.0|  394|
|            37.0|  537|
|            36.0|  862|
|            35.0|  824|
|            34.0|  689|
|            33.0|  615|
+----------------+-----+
only showing top 20 rows



In [22]:
df.describe().show()

+-------+-------------------+-----------------+------------------+------------------+-----------------+------------------+-----------------+------------------+------------------+
|summary|          longitude|         latitude|  housingMedianAge|        totalRooms|    totalBedrooms|        population|       households|      medianIncome|  medianHouseValue|
+-------+-------------------+-----------------+------------------+------------------+-----------------+------------------+-----------------+------------------+------------------+
|  count|              20640|            20640|             20640|             20640|            20640|             20640|            20640|             20640|             20640|
|   mean|-119.56970444871473|35.63186143109965|28.639486434108527|2635.7630813953488|537.8980135658915|1425.4767441860465|499.5396802325581|3.8706710030346416|206855.81690891474|
| stddev|  2.003531742932898|2.135952380602968| 12.58555761211163|2181.6152515827944| 421.247905943133|  

# Pré-processamento dos valores alvo

In [23]:
# A variável dependente deve ser normalizada
from pyspark.sql.functions import *

df = df.withColumn('medianHouseValue', col('medianHouseValue')/100000)

df.take(2)

[Row(longitude=-122.2300033569336, latitude=37.880001068115234, housingMedianAge=41.0, totalRooms=880.0, totalBedrooms=129.0, population=322.0, households=126.0, medianIncome=8.325200080871582, medianHouseValue=4.526),
 Row(longitude=-122.22000122070312, latitude=37.86000061035156, housingMedianAge=21.0, totalRooms=7099.0, totalBedrooms=1106.0, population=2401.0, households=1138.0, medianIncome=8.301400184631348, medianHouseValue=3.585)]

# Engenharia de Features

In [30]:
# # Dividindo o total de quartos pela quantidade de domicílios por grupo de blocos
# roomsPerHousehold = df.select(col('totalRooms')/col('households'))

# # Dividindo a população total por grupo de blocos pela quantidade de pessoas na familia
# populationPerHousehold = df.select(col('population')/col('households'))

# # Dividindo o total de quartos com cama pelo total de cômodos
# bedroomsPerRoom = df.select(col('totalBedRooms')/col('totalRooms'))

df = df.withColumn("roomsPerHousehold", col("totalRooms")/col("households")) \
   .withColumn("populationPerHousehold", col("population")/col("households")) \
   .withColumn("bedroomsPerRoom", col("totalBedRooms")/col("totalRooms"))

df.show(2)

+---------+--------+----------------+----------+-------------+----------+----------+------------+----------------+-----------------+----------------------+-------------------+
|longitude|latitude|housingMedianAge|totalRooms|totalBedrooms|population|households|medianIncome|medianHouseValue|roomsPerHousehold|populationPerHousehold|    bedroomsPerRoom|
+---------+--------+----------------+----------+-------------+----------+----------+------------+----------------+-----------------+----------------------+-------------------+
|  -122.23|   37.88|            41.0|     880.0|        129.0|     322.0|     126.0|      8.3252|           4.526|6.984126984126984|    2.5555555555555554|0.14659090909090908|
|  -122.22|   37.86|            21.0|    7099.0|       1106.0|    2401.0|    1138.0|      8.3014|           3.585|6.238137082601054|     2.109841827768014|0.15579659106916466|
+---------+--------+----------------+----------+-------------+----------+----------+------------+----------------+------

In [31]:
# Reordenando a ordem das colunas e retirando colunas indesejadas. A variável alvo ficará em primeiro para que não seja afetadan a padronização
df = df.select("medianHouseValue", 
              "totalBedRooms", 
              "population", 
              "households", 
              "medianIncome", 
              "roomsPerHousehold", 
              "populationPerHousehold", 
              "bedroomsPerRoom")

df.show(3)

+----------------+-------------+----------+----------+------------+-----------------+----------------------+-------------------+
|medianHouseValue|totalBedRooms|population|households|medianIncome|roomsPerHousehold|populationPerHousehold|    bedroomsPerRoom|
+----------------+-------------+----------+----------+------------+-----------------+----------------------+-------------------+
|           4.526|        129.0|     322.0|     126.0|      8.3252|6.984126984126984|    2.5555555555555554|0.14659090909090908|
|           3.585|       1106.0|    2401.0|    1138.0|      8.3014|6.238137082601054|     2.109841827768014|0.15579659106916466|
|           3.521|        190.0|     496.0|     177.0|      7.2574|8.288135593220339|    2.8022598870056497|0.12951601908657123|
+----------------+-------------+----------+----------+------------+-----------------+----------------------+-------------------+
only showing top 3 rows



# Padronização

In [34]:
from pyspark.ml.linalg import DenseVector

input_data = df.rdd.map(lambda x: (x[0], DenseVector(x[1:])))
df = spark.createDataFrame(input_data, ['label', 'features'])

In [40]:
df.show(3)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|4.526|[129.0,322.0,126....|
|3.585|[1106.0,2401.0,11...|
|3.521|[190.0,496.0,177....|
+-----+--------------------+
only showing top 3 rows



In [45]:
from pyspark.ml.feature import StandardScaler

standardScaler = StandardScaler(inputCol="features", outputCol="features_scaled")

scaler = standardScaler.fit(df)

scaled_df = scaler.transform(df)

scaled_df.take(2)

[Row(label=4.526, features=DenseVector([129.0, 322.0, 126.0, 8.3252, 6.9841, 2.5556, 0.1466]), features_scaled=DenseVector([0.3062, 0.2843, 0.3296, 4.3821, 2.8228, 0.2461, 2.5264])),
 Row(label=3.585, features=DenseVector([1106.0, 2401.0, 1138.0, 8.3014, 6.2381, 2.1098, 0.1558]), features_scaled=DenseVector([2.6255, 2.1202, 2.9765, 4.3696, 2.5213, 0.2031, 2.6851]))]

In [46]:
train_data, test_data = scaled_df.randomSplit([.8,.2],seed=1234)

In [47]:
from pyspark.ml.regression import LinearRegression

model = LinearRegression(labelCol='label', maxIter=10, regParam=0.3, elasticNetParam=0.8)

linear_model = model.fit(train_data)

In [51]:
predicted = linear_model.transform(test_data)

predictions = predicted.select("prediction").rdd.map(lambda x: x[0])
labels = predicted.select('label').rdd.map(lambda x: x[0])

predictionAndLabel = predictions.zip(labels).collect()
predictionAndLabel[:5]

[(1.4542345782162025, 0.14999),
 (1.6495895629633672, 0.175),
 (1.482098957606102, 0.332),
 (1.6123448501897313, 0.346),
 (1.6541552239238302, 0.35)]

In [53]:
linear_model.coefficients

DenseVector([0.0, 0.0, 0.0, 0.2767, 0.0, 0.0, 0.0])

In [54]:
linear_model.intercept

0.9947076240544817

In [56]:
linear_model.summary.rootMeanSquaredError

0.4192463677898063

In [57]:
linear_model.summary.r2

0.4192463677898063

In [58]:
# Parar a execução do spark
spark.stop()